In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

#try:
    # %tensorflow_version only exists in Colab.
#    %tensorflow_version 2.0
#except Exception:
#    pass

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix

tf.config.experimental.set_visible_devices([], 'GPU')

AUTOTUNE = tf.data.experimental.AUTOTUNE
import IPython.display as display


import os
from os import listdir
from os.path import isfile, join
import pickle
import random
#importing module
import sys
sys.path.insert(0, '../data')
from datahandler_multilabel import create_dataset

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt



print(tf.__version__)
#base_model = model.output

2.1.0


In [2]:
with open('../data/train_test_split_hierarchical.pkl', 'rb') as infile:
    train_x, train_y, train_y_, val_x, val_y, val_y_, test_x, test_y, test_y_ = pickle.load(infile)

In [3]:
train_generator_coarse = create_dataset(train_x, train_y)
val_generator_coarse = create_dataset(val_x, val_y)
test_generator_coarse = create_dataset(test_x, test_y, BATCH_SIZE=1)
last = [14, 21, 12, 14, 2, 16, 18, 3, 5, 14, 12, 11, 5, 7, 6]
discard = [34,47,113,117,118,146]
last = [14, 20, 11, 14, 2, 16, 18, 3, 5, 11, 12, 11, 4, 7, 6]
print(sum(last)-len(discard))
train_generator_fine = [0]*len(last)
val_generator_fine = [0]*len(last)
test_generator_fine = [0]*len(last)
for i in range(len(last)):
    train_generator_fine[i] = create_dataset(train_x, [elt[sum(last[:i]):sum(last[:i+1])] for elt in train_y_], BATCH_SIZE=32)
    val_generator_fine[i] = create_dataset(val_x, [elt[sum(last[:i]):sum(last[:i+1])] for elt in val_y_], BATCH_SIZE=32)
    test_generator_fine[i] = create_dataset(test_x, [elt[sum(last[:i]):sum(last[:i+1])] for elt in test_y_], BATCH_SIZE=1)

148


In [24]:
class Coarse(tf.keras.layers.Layer):
    def __init__(self, filters, shape=(224,224,3), kernel_size=3,
                 pool_size=(2,2), activation='relu', padding='same'):
        super(Coarse, self).__init__()
        self.input_layer = tf.keras.layers.InputLayer(
            #input_shape = shape,
        )
        self.conv_layer = tf.keras.layers.Conv2D(
            filters = filters[0], 
            kernel_size = kernel_size,
            activation = activation,
            )
        self.nn_pooling = tf.keras.layers.MaxPooling2D(
            pool_size = pool_size,
            padding = padding,
            )
        self.nn_dropout = tf.keras.layers.Dropout(0.2)
        self.nn_batchnorm = tf.keras.layers.BatchNormalization(axis=-1)
        self.flat = tf.keras.layers.Flatten()#input_shape = (13, 13, 64))
        self.dense = tf.keras.layers.Dense(filters[1], activation='relu')
        self.out = tf.keras.layers.Dense(filters[2], activation='softmax')

        
        
    def call(self, input_features):
        activation = self.input_layer(input_features)
        activation = self.conv_layer(activation)
        activation = self.nn_batchnorm(activation)
        activation = self.nn_pooling(activation)
        print(input_features.shape,activation.shape)
        activation = self.flat(activation)
        activation = self.dense(activation)
        return self.out(activation)


In [4]:
def macro_f1(y, y_hat, thresh=0.5):
    y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    macro_f1 = tf.reduce_mean(f1)
    return macro_f1

def accuracy(y, y_hat):
    y_pred = tf.cast(tf.greater(y_hat, 0.5), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    tn = tf.cast(tf.math.count_nonzero((1 - y_pred) * (1 - y), axis=0), tf.float32)
    accuracy = (tp + tn) / (tp + tn + fn + fp)
    accuracy = tf.reduce_mean(accuracy)
    return accuracy

def precision(y, y_hat):
    y_pred = tf.cast(tf.greater(y_hat, 0.5), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    pres = tp / (tp + fp + 1e-16)
    precision = tf.reduce_mean(pres)
    return precision

def recall(y, y_hat):
    y_pred = tf.cast(tf.greater(y_hat, 0.5), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    rec = tp / (tp + fn + 1e-16)
    recall = tf.reduce_mean(rec)
    return recall

In [5]:
file = 'training_flat_multilabelVGG_weight.h5'
model_c = tf.keras.models.load_model('../multi-label/results/'+file, compile=False,)

  
model_c.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=[accuracy, recall, precision, macro_f1])

for i in range(len(model_c.layers)-1):
    model_c.layers[i].set_weights(model_c.layers[i].get_weights())

In [ ]:
index = 30
step = 5
stop = 50

while index < stop:
    model_c.fit(train_generator_coarse,
          epochs=index+step,
          verbose=1, initial_epoch=index,
          validation_data = val_generator_coarse
          )
    index += step

In [6]:
#constructing fine classifiers
class Fine(tf.keras.layers.Layer):
    def __init__(self, filters, shape=(224,224,3), kernel_size=3,
                 pool_size=(2,2), activation='relu', padding='same'):
        super(Fine, self).__init__()
        self.input_layer = tf.keras.layers.InputLayer(
            #input_shape = shape,
        )
        self.conv_layer = tf.keras.layers.Conv2D(
            filters = filters[0], 
            kernel_size = kernel_size,
            activation = activation,
            )
        self.nn_pooling = tf.keras.layers.MaxPooling2D(
            pool_size = pool_size,
            padding = padding,
            )
        self.nn_dropout = tf.keras.layers.Dropout(0.2)
        self.nn_batchnorm = tf.keras.layers.BatchNormalization(axis=-1)
        self.flat = tf.keras.layers.Flatten()#input_shape = (13, 13, 64))
        self.dense = tf.keras.layers.Dense(filters[1], activation='relu')
        self.out = tf.keras.layers.Dense(filters[2], activation='softmax')

        
        
    def call(self, input_features):
        activation = self.input_layer(input_features)
        activation = self.conv_layer(activation)
        activation = self.nn_batchnorm(activation)
        activation = self.nn_pooling(activation)
        activation = self.flat(activation)
        activation = self.dense(activation)
        return self.out(activation)


def fine_model(last):
    model_fine = keras.Sequential([
        #base_model,
        Fine([64, 128, last])
    ])

  
    model_fine.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=[accuracy, recall, precision, macro_f1])

    for i in range(len(model_fine.layers)-1):
        model_fine.layers[i].set_weights(model_c.layers[i].get_weights())
    return model_fine

In [7]:
#last = [14, 21, 12, 14, 2, 16, 18, 3, 5, 14, 12, 11, 5, 7, 6]
fine_models = {'models' : [{} for i in range(15)], 'yhf' : [{} for i in range(2)]}

for i in range(15):
    model_i = fine_model(last[i])
    fine_models['models'][i] = model_i

In [8]:
def get_error(y,yh):
    # Threshold 
    yht = np.zeros(np.shape(yh))
    yht[np.arange(len(yh)), yh.argmax(1)] = 1
    # Evaluate Error
    error = np.count_nonzero(np.count_nonzero(y-yht,1))/len(y)
    return error

In [11]:
#training fine classifiers on corresponding data
for cat in range(13):
    index = 0
    step = 1
    stop = 3
    while index < stop:
        fine_models['models'][cat].fit(train_generator_fine[cat],
          epochs = index+step,
          verbose=1, initial_epoch=index,
          validation_data = val_generator_fine[cat]
          )
        index += step
    
    #yh_f = fine_models['models'][cat].predict(test_generator_fine[cat])
    print('Fine Classifier '+str(cat)+' Error: ')
    #str(get_error(y_val[ix_v],yh_f)))

Train for 563 steps, validate for 94 steps
563/563 [==============================] - 1233s 2s/step - loss: 1.6025 - accuracy: 0.8957 - recall: 0.0712 - precision: 0.0463 - macro_f1: 0.0502 - val_loss: 1.3938 - val_accuracy: 0.9093 - val_recall: 0.0714 - val_precision: 0.0402 - val_macro_f1: 0.0505
Train for 563 steps, validate for 94 steps
Epoch 2/2
563/563 [==============================] - 1192s 2s/step - loss: 1.5584 - accuracy: 0.8986 - recall: 0.0714 - precision: 0.0374 - macro_f1: 0.0475 - val_loss: 1.3943 - val_accuracy: 0.9092 - val_recall: 0.0714 - val_precision: 0.0394 - val_macro_f1: 0.0503
Train for 563 steps, validate for 94 steps
Epoch 3/3
563/563 [==============================] - 1347s 2s/step - loss: 1.5583 - accuracy: 0.8986 - recall: 0.0714 - precision: 0.0374 - macro_f1: 0.0475 - val_loss: 1.3944 - val_accuracy: 0.9092 - val_recall: 0.0714 - val_precision: 0.0394 - val_macro_f1: 0.0504
Fine Classifier 0 Error: 
Train for 563 steps, validate for 94 steps
563/563 [==

ResourceExhaustedError:  OOM when allocating tensor with shape[788544,128] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node Adam/Adam/update_4/ResourceApplyAdam (defined at <ipython-input-11-debd31c6b462>:10) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_distributed_function_32429]

Errors may have originated from an input operation.
Input Source operations connected to node Adam/Adam/update_4/ResourceApplyAdam:
 sequential_2/fine_2/dense_4/MatMul/ReadVariableOp (defined at <ipython-input-6-bbac023ba889>:32)

Function call stack:
distributed_function


In [ ]:
coarse_predictions=model_c.predict(test_generator_coarse)

fine_predictions = [] #dim:  n_classes_coarse X n_images_predict X n_classes_fine
for c in range(2):
    fine_predictions.append(fine_models['models'][c].predict(test_generator_fine[c]))

In [ ]:
prediction_size = len(coarse_predictions)
predictions = []
for img in range(prediction_size):
    proba = [0]*10
    for finec in range(10):
        for coarsec in range(2):
            proba[finec] += coarse_predictions[img][coarsec]*fine_predictions[coarsec][img][finec]
    predicted = np.argmax(proba)
    predictions.append(predicted)

In [9]:
def eval_hdcnn(X, y):
    yh = np.zeros(np.shape(y))
    
    yh_s = model.predict(X, batch_size=32)
    
    print('Single Classifier Error: '+str(get_error(y,yh_s)))
    
    yh_c = model_c.predict(X, batch_size=32)
    y_c = np.dot(y,fine2coarse)
    
    print('Coarse Classifier Error: '+str(get_error(y_c,yh_c)))

    for i in range(2):
        if i%5 == 0:
            print("Evaluating Fine Classifier: ", str(i))
        #fine_models['yhf'][i] = fine_models['models'][i].predict(X, batch_size=batch)
        yh += np.multiply(yh_c[:,i].reshape((len(y)),1), fine_models['yhf'][i])
    
    print('Overall Error: '+str(get_error(y,yh)))
    return yh